# Práctica 8
# Regresión lineal, multiple y logística

### Santiago Blasco Arnaiz

## Apartado 1

### Comandos bash para la descarga, descompresión y creación de los ficheros requeridos

### (Las siguientes celdas no serán ejecutadas, para que se ejecuten han de cambiarse a celdas de código)

### (Aquí terminan las celdas que ejecutan comandos bash)

In [1]:
import pandas as pd

In [2]:
dataRaw = pd.read_csv('dataset.txt')

### Dataset

In [3]:
dataRaw

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,Class
0,0.266074,-0.165620,0.320980,0.483299,0.172340,0.273364,0.371178,0.929823,0.251173,0.159345,A
1,0.130292,0.870736,-3.210528,0.062493,0.261718,1.436060,1.465940,0.636203,0.282354,0.515587,A
2,-0.116585,0.069915,0.068476,-0.783147,0.261718,0.439463,-0.081827,-0.888236,-0.123005,0.582939,A
3,0.031541,0.297600,-3.210528,-0.583590,-0.721442,-0.307984,0.710932,1.051693,0.594169,-0.533994,A
4,0.229043,0.807926,-0.052442,0.082634,0.261718,0.148790,0.635431,0.051062,0.032902,-0.086652,F
...,...,...,...,...,...,...,...,...,...,...,...
20862,-0.128929,-0.040001,0.057807,0.557894,0.261718,-0.930856,-0.044076,1.158458,2.277968,-0.699884,X
20863,0.266074,0.556689,-0.020434,0.176624,0.261718,-0.515608,0.597681,0.178349,0.625350,-0.657245,G
20864,-0.054866,0.580242,0.032912,-0.016668,0.261718,1.519109,0.371178,-0.985508,-0.403638,1.276301,A
20865,0.080916,0.588093,0.015130,0.002250,0.261718,-0.930856,-0.270579,0.163807,-0.091823,-0.593329,F


## Apartado 2

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [5]:
#Metemos el dataframe en un array para poder trabajar más comodamente
data = np.asarray(dataRaw)

#Separamos el conjunto en los valores de las etiquetas y de la clae
XRaw,Y = data[:,:-1], data[:,-1]

#Creamos el escalador y escalamos los datos
scaler = MinMaxScaler()
X = scaler.fit_transform(XRaw)

In [6]:
#Binarizamos los posibles valores de la clase
binTarget = LabelBinarizer().fit(np.unique(Y)).transform(Y)
binTarget.shape

(20867, 12)

### Hold out

In [7]:
#Separamos tanto los valores de las etiquetas como de la clase
#en los conjuntos de train (2/3) y test (1/3)
X_train,X_test,Y_train,Y_test = train_test_split(X,binTarget, test_size= 1/3, random_state=1, stratify=Y)

X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((13911, 10), (6956, 10), (13911, 12), (6956, 12))

In [8]:
#Creamos el array en el que almacenaremos la predicción de cada clasificador
Y_predict = np.zeros(Y_test.shape, dtype=float)

#Necesitamos un clasificador para cada valor de la clase
for i in range(Y_test.shape[1]):
    
    #Creamos el clasificador y lo ajustamos con los datos de entrenamiento de la clase que toca
    regresion = LinearRegression().fit(X_train, Y_train[:,i])
    
    #Añadimos la predicción al array de predicciones
    Y_predict[:,i] = regresion.predict(X_test)

### Tasa de aciertos obtenida con hold out utilizando regresión lineal de respuesta múltiple

In [9]:
print("Tasa de aciertos =", accuracy_score(np.argmax(Y_test, axis=1), np.argmax(Y_predict, axis=1)))

Tasa de aciertos = 0.4778608395629672


### Matriz de confusión obtenida con hold out utilizando regresión lineal de respuesta múltiple

In [10]:
print("\nMatriz de Confusión:\n", confusion_matrix(np.argmax(Y_test, axis=1), np.argmax(Y_predict, axis=1)))


Matriz de Confusión:
 [[2780    0    0    0    3   37    0    0   37    0    0    0]
 [   0    0    0    0    0    0    0    0    3    0    0    0]
 [  55    0    0    0    2    7    0    0    5    0    0    0]
 [ 220    0    0    0    5    2    0    0    8    0    0    0]
 [ 675    0    0    0   12   29    0    0   14    0    0    0]
 [1285    0    0    0    0    8    0    0   15    0    0    0]
 [ 289    0    0    0    1    6    0    0    2    0    0    0]
 [ 276    0    0    0    7   30    0    0   33    0    0    0]
 [  55    0    0    0    3   10    0    0  482    0    4    0]
 [  26    0    0    0    4    0    0    0    0    0    0    0]
 [ 211    0    0    0   12    0    0    0   83    0   42    0]
 [  37    0    0    0    5    1    0    0  131    0    4    0]]


## Apartado 3

### Cross validation 10 particiones

In [11]:
from sklearn.model_selection import StratifiedKFold

#Número de particiones en que queremos dividir el dataset
subsamples = 10

skf = StratifiedKFold(n_splits=subsamples, random_state=None, shuffle=False)
#Variables para calcular la tasa de acierto media
score = 0
#Asignamos las particiones de train y test un total de 10 veces
for train_index, test_index in skf.split(X, Y):
    #Dividimos en train y test según los índices que nos marca el iterador
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = binTarget[train_index], binTarget[test_index]
    
    #Creamos el array en el que almacenaremos la predicción de cada clasificador
    Y_predict = np.zeros(Y_test.shape, dtype=float)
    
    #Necesitamos un clasificador para cada valor de la clase
    for i in range(Y_test.shape[1]):
        
        #Creamos el clasificador y lo ajustamos con los datos de entrenamiento de la clase que toca
        regresion = LinearRegression().fit(X_train, Y_train[:,i])
        
        #Añadimos la predicción al array de predicciones
        Y_predict[:,i] = regresion.predict(X_test)
    
    #Sumamos todos los scores
    score = score + accuracy_score(np.argmax(Y_test, axis=1), np.argmax(Y_predict, axis=1))

#Para a continuación obtener el score medio
score = score/subsamples

### Tasa de aciertos obtenida con XV 10 particiones utilizando regresión lineal de respuesta múltiple

In [12]:
print("Tasa de aciertos =", score)

Tasa de aciertos = 0.4814777688296402


### Comparación de resultados

Ambos resultados son similares, no notamos una mejora significativa al variar los conjuntos de entrenamiento y test.

## Apartado 4

In [13]:
from sklearn.linear_model import LogisticRegression

### Hold out

In [14]:
#Separamos tanto los valores de las etiquetas como de la clase
#en los conjuntos de train (2/3) y test (1/3)
X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size= 1/3, random_state=1, stratify=Y)

X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((13911, 10), (6956, 10), (13911,), (6956,))

In [15]:
#Creamos el clasificador
#(El parámetro multi_class lo dejamos por defecto ya que 
#entonces toma el valor 'ovr' que lo ajusta a un problema binario por cada etiqueta de clase)
clf = LogisticRegression(n_jobs=-1)

#Ajustamos el clasificador con los datos de entrenamiento
clf.fit(X_train, Y_train)

#Predecimos con el clasificador que acabamos de crear y ajsutar
Y_predict = clf.predict(X_test)

#Obtenemos la tasa de acierto de lo predicho respecto a lo esperado
score = clf.score(X_test, Y_test)

### Tasa de aciertos obtenida con hold out utilizando regresión logística multiclase

In [16]:
print("Tasa de aciertos =", score)

Tasa de aciertos = 0.49511213341000576


### Matriz de confusión obtenida con hold out utilizando regresión logística multiclase

In [17]:
print("\nMatriz de Confusión:\n", confusion_matrix(Y_test, Y_predict))


Matriz de Confusión:
 [[2804    0    0    0    6   45    0    0    0    0    2    0]
 [   3    0    0    0    0    0    0    0    0    0    0    0]
 [  62    0    0    0    0    7    0    0    0    0    0    0]
 [ 226    0    0    0    0    8    0    1    0    0    0    0]
 [ 711    0    0    0    0   19    0    0    0    0    0    0]
 [1290    0    0    0    4    7    0    3    4    0    0    0]
 [ 293    0    0    0    0    0    0    1    0    0    4    0]
 [ 305    0    0    0    0   35    0    0    6    0    0    0]
 [  47    0    0    0    1    6    0    7  490    0    3    0]
 [  30    0    0    0    0    0    0    0    0    0    0    0]
 [ 198    0    0    0    3    4    0    0    0    0  143    0]
 [ 120    0    0    0    2    0    0    0   25    0   31    0]]


### Cross validation 10 particiones

In [18]:
#Número de particiones en que queremos dividir el dataset
subsamples = 10

skf = StratifiedKFold(n_splits=subsamples, random_state=None, shuffle=False)
#Variable para calcular la tasa de acierto media
score = 0
#Asignamos las particiones de train y test un total de 10 veces
for train_index, test_index in skf.split(X, Y):
    #Dividimos en train y test según los índices que nos marca el iterador
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    #Creamos el clasificador
    clf = LogisticRegression(n_jobs=-1)
    
    #Ajustamos el clasificador con los datos de entrenamiento
    clf.fit(X_train, Y_train)
    
    #Sumamos todos los scores
    score = score + clf.score(X_test, Y_test)

#Para a continuación obtener el score medio
score = score/subsamples

### Tasa de aciertos obtenida con XV 10 particiones utilizando regresión logística multiclase

In [19]:
print("Tasa de aciertos =", score)

Tasa de aciertos = 0.49753192961404225


### Comparación de resultados

Ambos resultados son prácticamente idénticos, no notamos una mejora significativa al variar los conjuntos de entrenamiento y test. Respecto a los resultados obtenidos con la regresión lineal múltiple tampoco cambian mucho además estos eran menos costosos computacionalmente, la validación cruzada con regresión logística puede ser paralelizada aunque en nuestro caso no sea así y notemos ese aumento del coste computacional que mencionamos.